In [7]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import torch
import cv2
import scipy.signal
import scipy.ndimage
import sys
import os
os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"
sys.path.append("../../")
import helpers
import glm

%load_ext autoreload
%autoreload 2

In [5]:
# TODO: UNUSED CODE, IGNORE THIS CELL
# # Just to visualize images
# fig, ax = plt.subplots()
# ax.axis("off")
# ax.set_title("Projected points")
# ax.imshow(windowed(testing_0, 8).reshape((128,128)), cmap="gray")
# # ax.imshow(testing_0, cmap="gray")
# 
# fig.tight_layout()
# plt.show()
# print(testing_0)

#     # return img.reshape((int(img.shape[0] / window_size), int(img.shape[1] / window_size), window_size, window_size))
#     
# def windowed(img, window_size):
#     return ressample(img, window_size)
# def ressample(arr, N):
#     A = []
#     for v in np.vsplit(arr, arr.shape[0] // N):
#         A.extend([*np.hsplit(v, arr.shape[0] // N)])
#     return np.array(A)
# 
# 
        # up_offs_row = max(row - window_size // (window_size - overlap) - 1, 0)
        # down_offs_row = min(row + window_size // (window_size - overlap) + 1, window_dim)
        # left_offs_col = max(col - window_size // (window_size - overlap) - 1, 0)
        # right_offs_col = min(col + window_size // (window_size - overlap) + 1, window_dim)
        # current_wind_2 = img2_4d[left_offs_col:right_offs_col, up_offs_row:down_offs_row, :, :]

4tw 5
frgetsdgd 5 fdsrfrs


In [8]:
# WINDOWING FUNCTIONS
# assuming for now that window size fits perfectly into img, and img is 2d np array perfect square  
def windowed_old(img, window_size):
    return np.lib.stride_tricks.as_strided(img, shape=(int(img.shape[0] / window_size), int(img.shape[1] / window_size), window_size, window_size), strides =(img.strides[0] * window_size, img.strides[1] * window_size, img.strides[0], img.strides[1]) , writeable=False)


In [9]:
def moving_window_array(array: torch.Tensor, window_size, overlap) -> torch.Tensor:
    shape = array.shape
    strides = (
        shape[-1] * (window_size - overlap),
        (window_size - overlap),
        shape[-1],
        1
    )
    shape = (
        int((shape[-2] - window_size) / (window_size - overlap)) + 1,
        int((shape[-1] - window_size) / (window_size - overlap)) + 1,
        window_size,
        window_size,
    )
    return torch.as_strided(
        array, size=shape, stride=strides)

In [ ]:
#### OLD VERSION ###
# img2_4d = img2_windowed.reshape(window_dim, window_dim, img2_windowed.shape[-2], img2_windowed.shape[-1])
#     vector_field_shape = list((*img2_4d.shape[:2], 2))
#     vector_field = np.zeros(vector_field_shape)
#     for i, window_1 in enumerate(img1_windowed):
#         row = i // window_dim
#         col = i % window_dim
#         rel_windows_2 = img2_4d[max((row-4),0):min((row+4) + 1,window_dim), max((col-4),0):min((col+4) + 1,window_dim), :, : ]
#         rel_diffs = np.zeros(rel_windows_2.shape[:2])
#         for j, rel_cols_2 in enumerate(rel_windows_2):
#             for k, rel_wind_2 in enumerate(rel_cols_2):
#                 rel_diffs[j,k] = torch.sum(torch.abs(window_1 - rel_wind_2)).item() / (window_size ** 2)
#         
#         vector_field[row, col, :] = index_min(rel_diffs)
#     return np.array(vector_field)
# bc = calculate(testing_0, testing_1, 64, 61)

In [212]:
# testtesttorch = torch.from_numpy(testtest)
# print(testtesttorch)
# print(testtesttorch.shape)
# print(moving_window_array(testtesttorch, 4, 3).shape)

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7],
        [ 8,  9, 10, 11, 12, 13, 14, 15],
        [16, 17, 18, 19, 20, 21, 22, 23],
        [24, 25, 26, 27, 28, 29, 30, 31],
        [32, 33, 34, 35, 36, 37, 38, 39],
        [40, 41, 42, 43, 44, 45, 46, 47],
        [48, 49, 50, 51, 52, 53, 54, 55],
        [56, 57, 58, 59, 60, 61, 62, 63]], dtype=torch.int32)
5
torch.Size([25, 4, 4])


In [10]:
def index_min(lst):
    index = lst.argmin()
    col = lst.shape[1]
    return index//col, index % col  # row, col

In [15]:
testing_0 = helpers.imread_normalized_float_grayscale(os.path.join("test", "0.jpg"))
testing_0 = np.where(testing_0 > 0.5, 1.0, 0.0)  # TEMPORARY, TO MAKE TEST MORE IDEAL
testing_1 = helpers.imread_normalized_float_grayscale(os.path.join("test", "1.jpg"))
testing_1 = np.where(testing_1 > 0.5, 1.0, 0.0)  # TEMPORARY, TO MAKE TEST MORE IDEAL
testtest_0 = np.array(np.split(np.arange(64), 8))
testtest_1 = np.array(np.split(np.arange(64), 8)) - 1

In [27]:
# print(testtest_0)
print(testtest_0)
print(moving_window_array(torch.from_numpy(testtest_0), 4, 3).shape)

[[ 0  1  2  3  4  5  6  7]
 [ 8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23]
 [24 25 26 27 28 29 30 31]
 [32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47]
 [48 49 50 51 52 53 54 55]
 [56 57 58 59 60 61 62 63]]
torch.Size([5, 5, 4, 4])


In [472]:
# Assumptions for simple version: ignoring restrictions and bug checks, assuming window size fits into img with no remainder, overlap is strictly less than window size, images are numpy arrays and perfect squares
def calculate(img1, img2, window_size, overlap):
    # split img1 into windows, 3d array, with all possible overlaps:
    img1_windowed = moving_window_array(torch.from_numpy(img1), window_size, overlap)
    img2_windowed = moving_window_array(torch.from_numpy(img2), window_size, overlap)
    # Check correlation from 1 image to next one, for each window, to get movement of each patch.
    # Searching whole image for correlation for each window is redundant, only searching for two windows across. Assumption: Each image split into at least a 2x2 grid of windows.
    offset = 4  # for now
    window_dim = int(img2_windowed.shape[0])
    vector_field_shape = list((*img2_windowed.shape[:2], 2))
    vector_field = np.zeros(vector_field_shape)
    for row_1, window_1_row in enumerate(img1_windowed):
            for col_1, window_1 in enumerate(window_1_row):
                row_min = max((row_1-offset), 0)
                row_max = min((row_1+offset) + 1, window_dim)
                col_min = max((col_1-offset), 0)
                col_max = min((col_1+offset) + 1, window_dim)
                rel_windows_2 = img2_windowed[row_min:row_max, col_min:col_max, :, : ]
                rel_diffs = np.zeros(rel_windows_2.shape[:2])
                for row_2, rel_row_2 in enumerate(rel_windows_2):
                    for col_2, rel_wind_2 in enumerate(rel_row_2):
                        rel_diffs[row_2,col_2] = torch.sum(torch.abs(window_1 - rel_wind_2)).item() / (rel_windows_2.shape[0] * rel_windows_2.shape[1])
                smallest_rel_index = index_min(rel_diffs)
                wind_2_abs_loc = np.array((row_min, col_min)) + smallest_rel_index
                vect = wind_2_abs_loc - (row_1, col_1)
                vector_field[row_1, col_1, :] = vect
    return np.array(vector_field)
bc = calculate(testing_0, testing_1, 32, 31)


In [500]:
print(bc[10,10])  # correct, the img moves 3 pixels to the right. at the right edge it goes off the image, so there the arrows do weird stuff, which is fine.

[0. 3.]


In [45]:
mm = [["a", "b"], ["c", "d"], ["e", "f"], ["g", "h"]]
for index_1, element_1 in enumerate(mm):
    for index_2, element_2 in enumerate(element_1):
        print(index_1, element_2)
        # print(index_1, index_2, element_2)
    


0 a
0 b
1 c
1 d
2 e
2 f
3 g
3 h
